In [1]:
import os

In [2]:
!pwd

/Users/vedantmahida/Desktop/end-to-end-ml-with-mlflow/notebooks


In [3]:
os.chdir('../')

In [4]:
!pwd

/Users/vedantmahida/Desktop/end-to-end-ml-with-mlflow


In [5]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/mahidavedant/end-to-end-ml-with-mlflow.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'mahidavedant'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '19390786719f78c7b137e4079a8efcd41c8e84ac'

In [11]:
# 1. Update config.yaml
# 4. Update entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [12]:
# 5. Update configuration manager in src config
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH

(PosixPath('config/config.yaml'),
 PosixPath('params.yaml'),
 PosixPath('schema.yaml'))

In [13]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/mahidavedant/end-to-end-ml-with-mlflow.mlflow",

        )

        return model_evaluation_config

In [14]:
# 6. Update components
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn 
import numpy as np
import joblib


class ModelEvaluation:
  def __init__(self, config: ModelEvaluationConfig):
    self.config = config


  def eval_metrics(self, y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2


  def log_into_mlflow(self):
    test_data = pd.read_csv(self.config.test_data_path)
    model = joblib.load(self.config.model_path)

    X_test = test_data.drop([self.config.target_column], axis=1)
    y_test = test_data[[self.config.target_column]]

    # set mlflow registery and parse mlflow tracking uri
    mlflow.set_registry_uri(self.config.mlflow_uri)
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Start run
    with mlflow.start_run():
      predicted_quantities = model.predict(X_test)
      (rmse, mae, r2) = self.eval_metrics(y_test, predicted_quantities)

      # saving metrics to local 
      scores = {'rmse': rmse, 'mae': mae, 'r2': r2}
      save_json(path=Path(self.config.metric_file_name), data=scores)

      # log params and metrics to mlflow
      mlflow.log_params(self.config.all_params)
      mlflow.log_metric('rmse', rmse)
      mlflow.log_metric('mae', mae)
      mlflow.log_metric('r2', r2)

      ####
      if tracking_url_type_store != 'file':
        mlflow.sklearn.log_model(model, 'model', registered_model_name='ElasticNetModel')
      else:
        mlflow.sklearn.log_model(model, 'model')



In [15]:
# 7. Update pipeline
try:
  config = ConfigurationManager()
  model_eval_config = config.get_model_evaluation_config()
  model_eval = ModelEvaluation(config=model_eval_config)
  model_eval.log_into_mlflow()
except Exception as e:
  raise e

Mar 21 09:14:38 common:65 INFO YAML file: config/config.yaml loaded successfully!
Mar 21 09:14:38 common:65 INFO YAML file: params.yaml loaded successfully!
Mar 21 09:14:38 common:65 INFO YAML file: schema.yaml loaded successfully!
Mar 21 09:14:38 common:29 INFO Directory created at: artifacts
Mar 21 09:14:38 common:29 INFO Directory created at: artifacts/model_evaluation
Mar 21 09:14:39 common:87 INFO JSON file saved at: artifacts/model_evaluation/metrics.json


Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2024/03/21 09:14:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNetModel, version 2
Created version '2' of model 'ElasticNetModel'.
